In [110]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
# Import libraries to aid your analysis
from bokeh.plotting import figure, output_notebook, show, output_file #builds interactive graphs for python
from bokeh.models import Range1d
import math #this is used in graphic section to use the irrational number pi
output_notebook() #Allows inline plotting for Juptyer notebook


Loading BokehJS ...

In [111]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=562 <class 'str'>


In [112]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [113]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [114]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress

starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1401


In [115]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1401


In [116]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

data_id  iso event_id_cnty event_id_no_cnty  event_date  year  \
0  7228161  562       NIR1452             1452  2020-09-18  2020   
1  7228145  562       NIR1450             1450  2020-09-18  2020   
2  7228153  562       NIR1453             1453  2020-09-18  2020   
3  7228156  562       NIR1449             1449  2020-09-18  2020   
4  7228158  562       NIR1451             1451  2020-09-18  2020   

  time_precision                  event_type                  sub_event_type  \
0              1  Violence against civilians  Abduction/forced disappearance   
1              1  Explosions/Remote violence   Remote explosive/landmine/IED   
2              1  Violence against civilians  Abduction/forced disappearance   
3              1  Explosions/Remote violence   Remote explosive/landmine/IED   
4              1  Violence against civilians  Abduction/forced disappearance   

                                              actor1  ...     location  \
0  Islamic State (West Africa) and/or Boko Haram ...  ...      Waragou   
1  Islamic State (West Africa) and/or Boko Haram ...  ...        Bosso   
2  Islamic State (West Africa) and/or Boko Haram ...  ...        Youwa   
3  Islamic State (West Africa) and/or Boko Haram ...  ...  Baroua Gana   
4  Islamic State (West Africa) and/or Boko Haram ...  ...    Makintari   

  latitude longitude geo_precision    source source_scale  \
0  13.5316   12.7309             1  Whatsapp    New media   
1  13.6987   13.3095             1  Whatsapp    New media   
2  13.4671   12.8197             1  Whatsapp    New media   
3  13.8986   13.1544             1  Whatsapp    New media   
4  13.5337   12.8181             1  Whatsapp    New media   

                                               notes fatalities   timestamp  \
0  On 18 September 2020, presumed ISWAP or Boko H...          0  1600717738   
1  On 18 September 2020, a Nigerien army vehicle ...          1  1600717738   
2  On 18 September 2020, presumed ISWAP or Boko H...          0  1600717738   
3  On 18 September 2020, a Nigerien army vehicle ...          0  1600717738   
4  On 18 September 2020, presumed ISWAP or Boko H...          0  1600717738   

  iso3  
0  NER  
1  NER  
2  NER  
3  NER  
4  NER  

[5 rows x 31 columns]

### Do the right thing, take some time to look at the codebook and see what these columns are

In [117]:
niger_res.columns

Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [118]:
#PIKE Checking the violent groups in Niger
niger_res.actor1

0       Islamic State (West Africa) and/or Boko Haram ...
1       Islamic State (West Africa) and/or Boko Haram ...
2       Islamic State (West Africa) and/or Boko Haram ...
3       Islamic State (West Africa) and/or Boko Haram ...
4       Islamic State (West Africa) and/or Boko Haram ...
                              ...                        
1396                 Military Forces of Niger (1996-1999)
1397                                   Protesters (Niger)
1398                                      Rioters (Niger)
1399                                      Rioters (Niger)
1400                        Tuareg Ethnic Militia (Niger)
Name: actor1, Length: 1401, dtype: object

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [119]:
!pip install pyproj
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
title_provider = get_provider('STAMEN_TERRAIN')

In [120]:
#PIKE Visualizing the groups and the geo coordiantes
country_map = niger_res[["latitude", 'longitude', 'actor1']]
country_map.head(7)

latitude longitude                                             actor1
0  13.5316   12.7309  Islamic State (West Africa) and/or Boko Haram ...
1  13.6987   13.3095  Islamic State (West Africa) and/or Boko Haram ...
2  13.4671   12.8197  Islamic State (West Africa) and/or Boko Haram ...
3  13.8986   13.1544  Islamic State (West Africa) and/or Boko Haram ...
4  13.5337   12.8181  Islamic State (West Africa) and/or Boko Haram ...
5  13.5997    6.4837                   Unidentified Armed Group (Niger)
6  14.2495   13.1092  Islamic State (West Africa) and/or Boko Haram ...

In [121]:
transformer = Transformer.from_crs('epsg:4326','epsg:3857')

In [122]:
map_dict = {} # empty dictionary to track group attacks by lat long
nan_count = {} # some data doesn't have a lat/long so we need to know what we are losing


# Iterate through tables and associate group with lat/long
for idx, row in country_map.iterrows():    
    if row['actor1'] in map_dict.keys(): 
        if math.isnan(float(row["latitude"])):
            #This counts no data
            if row['actor1'] in nan_count.keys(): 
                nan_count[row['actor1']] += 1 
            else: 
                nan_count[row['actor1']] = 1
        else: 
            #This has to convert the lat/long to a mercator projection 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['actor1']].append([point[0],point[1]]) 
    #BOTH the if an else statement do the same thing but since it is a dictionary one needs to add the group name first
    else:  
        if math.isnan(float(row["latitude"])):
          nan_count[row['actor1']] = 1
        else: 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['actor1']] =[[point[0],point[1]]]
    
        
#This tells how many attacks we are losing
nan_count

{}

In [123]:
#PIKE Displaying all the groups that produces violent events in Niger
for key in map_dict:
    print(key)

Islamic State (West Africa) and/or Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai wal-Jihad
Unidentified Armed Group (Niger)
Military Forces of France (2017-)
Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai wal-Jihad
JNIM: Group for Support of Islam and Muslims and/or Islamic State (Greater Sahara)
Islamic State (West Africa)
Fulani Ethnic Militia (Niger)
Rioters (Niger)
UFPR: Union of Patriotic Forces for the Refoundation of the Republic
Military Forces of Niger (2011-)
Government of Niger (2011-)
Military Forces of the United States (2017-)
Islamic State (Greater Sahara)
Police Forces of Niger (2011-) National Guard
Islamist Militia (Niger)
Buduma Ethnic Militia (Niger)
Military Forces of Niger (2011-) Gendarmerie
Protesters (Niger)
Unidentified Armed Group (Nigeria)
Rioters (Nigeria)
JNIM: Group for Support of Islam and Muslims
Rioters (International)
Military Forces of Chad (1990-)
Police Forces of Niger (2011-)
MNJTF: Multinational Joint Task Force
Kiki Communal Militia (Niger)
Lam

In [124]:
#PIKE Mapping the attacks from the first 5 top groups, defining the geo Coords for Niger
pts = [(25.00, 0), (10, 18.00)]
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)

In [125]:
#PIKE Visualizing the top 5 groups and the number of violent attacks per each
length_dict = {key: len(value) for key, value in map_dict.items()}

list_dict = [(k, v) for k, v in length_dict.items()]
list_dict.sort(key=lambda x: x[1], reverse=True)
for a in list_dict[0:5]:
    for items in a:
        print(items)

Islamic State (West Africa) and/or Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai wal-Jihad
205
Islamic State (Greater Sahara)
149
Unidentified Armed Group (Niger)
145
Islamic State (West Africa)
135
Protesters (Niger)
128


In [126]:
#PIKE Transferring the attackas of each of the identified 5 top groups into the transformed coordinates
for pt in map_dict["Islamic State (West Africa) and/or Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai wal-Jihad"]:
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])
for pt in map_dict["Islamic State (Greater Sahara)"]:
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])
for pt in map_dict["Unidentified Armed Group (Niger)"]:
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])
for pt in map_dict["Islamic State (West Africa)"]:
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])
for pt in map_dict["Protesters (Niger)"]:
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])


In [127]:
#Plots the bounding box
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator", title="VIOLENT EVENTS PER THE TOP FIVE GROUPS IN NIGER")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(title_provider)
# Places a circle for each converted lat/long attack 
p.circle(x = NPA_x, y = NPA_y, color= "firebrick")
#shows the plot
show(p)